4


In [7]:
b=[]
#hola
aux=[]
#parametros
nodos=[1,2,3]
rutas=[(1,1,1)]
numNodos=len(nodos)
numRutas=len(rutas)

#contruir b
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     b.append(aux[:])  
aux=1        
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            b[i][j]=50
            aux=aux+1
c=[]
aux=[]
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     c.append(aux[:])  
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            c[i][j]=abs(i-j)*10-abs(i-j)+1 
            
B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
for i in range(numNodos):
    for j in range(numNodos):
        B2[i,j,i]=b[i][j]
        B2[i,j,j]=-b[i][j]


arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]
rutasmodificadas=[]
rutaaux=[]
for i in range(numRutas):
   rutaaux.clear() 
   for j in range(numNodos):
        if rutas[i][j] == 1:
            rutaaux.append(j)          
   rutasmodificadas.append(rutaaux[:])  

#creacion parametro rkij


r={}
for i in arcos1:
    r[i]=0
for i in range(numRutas):
    for j in range(len(rutasmodificadas[i])-1):
        r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
        r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
#-------------------------
        
arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
arcosY=[(i) for i in range (numRutas)]
u= 100
m= 40
Q= 50
alpha=600

In [12]:

from docplex.mp.model import Model


mdl= Model("BRT")
#creacion variables de desicion
x=mdl.continuous_var_dict(arcosX,name="x")
y=mdl.continuous_var_dict(arcosY,name="y")


 # funcion a minimizar
costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
costoY=mdl.sum(alpha*(y[i])for i in range(numRutas))
obj=mdl.minimize(costoX+costoY)



#capacidad  
capacidad=[]
capacidadAux=[]
for i in  range(numNodos):
    for j in range(numNodos): 
        capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))
    
#flujo
for i in range(numNodos):  
    for o in range(numNodos):
        for d in range(numNodos):       
            mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)])
#maximo rutas
 
sumaRutas= mdl.sum(y[(k)]for k in range(numRutas))
maxRutas=mdl.add_constraint(sumaRutas<=m)  


mdl.solve(log_output=True)
mdl.report()








Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 19 rows and 64 columns.
Aggregator did 18 substitutions.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.04 ticks)
* model BRT solved with objective = 4600.000


In [9]:
z=mdl._dual_value1(maxRutas)
print(z)

0


In [14]:
for i in range(numRutas):
    if(y[(i)].solution_value):
        print(rutas[i],y[(i)].solution_value)

(1, 1, 1) 1.0


In [79]:
matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
r={}
for i in matriz:
    r[i]=0
    



mdlAux= Model("Aux")
rutaNueva=mdlAux.binary_var_dict(matriz,name="r")

dualCapacidad = mdl.dual_values(capacidad)



objAux=mdlAux.maximize(mdlAux.sum((-rutaNueva[i,j])*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))


mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=1)

for i in range(1,numNodos):    
    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1)))
    
for i in range(numNodos):
    for j in range(i+1,numNodos):
        mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])
        
for i in range(numNodos):
    mdlAux.add_constraint(rutaNueva[(i,i)]==0)
                       
mdlAux.solve(log_output=True)
mdlAux.report()                   

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 7 rows and 7 columns.
Aggregator did 2 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)
* model Aux solved with objective = 6.000


In [58]:

for i in range(numNodos):
    for j in range (numNodos):
        print(rutaNueva[(i,j)].solution_value,end=" ")
    print(" ")    

0 0 1.0  
0 0 0  
1.0 0 0  


In [59]:
b=[]
aux=[]
#parametros
nodos=[1,2,3]
rutas=[(1,1,1),(1,0,1)]
numNodos=len(nodos)
numRutas=len(rutas)

#contruir b
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     b.append(aux[:])  
aux=1        
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            b[i][j]=50
            aux=aux+1
c=[]
aux=[]
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     c.append(aux[:])  
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            c[i][j]=abs(i-j)*10-abs(i-j)+1 
            
B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
for i in range(numNodos):
    for j in range(numNodos):
        B2[i,j,i]=b[i][j]
        B2[i,j,j]=-b[i][j]


arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]
rutasmodificadas=[]
rutaaux=[]
for i in range(numRutas):
   rutaaux.clear() 
   for j in range(numNodos):
        if rutas[i][j] == 1:
            rutaaux.append(j)          
   rutasmodificadas.append(rutaaux[:])  

#creacion parametro rkij


r={}
for i in arcos1:
    r[i]=0
for i in range(numRutas):
    for j in range(len(rutasmodificadas[i])-1):
        r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
        r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
#-------------------------
        
arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
arcosY=[(i) for i in range (numRutas)]
u= 100
m= 40
Q= 50
alpha=600

from docplex.mp.model import Model


mdl= Model("BRT")
#creacion variables de desicion
x=mdl.continuous_var_dict(arcosX,name="x")
y=mdl.continuous_var_dict(arcosY,name="y")


 # funcion a minimizar
costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
costoY=mdl.sum(alpha*(y[i])for i in range(numRutas))
obj=mdl.minimize(costoX+costoY)



#capacidad  
capacidad=[]
capacidadAux=[]
for i in  range(numNodos):
    for j in range(numNodos): 
        capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))
    
#flujo
for i in range(numNodos):  
    for o in range(numNodos):
        for d in range(numNodos):       
            mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)])
#maximo rutas
 
sumaRutas= mdl.sum(y[(k)]for k in range(numRutas))
maxRutas=mdl.add_constraint(sumaRutas<=m)  


mdl.solve(log_output=True)
mdl.report()



Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 27 columns.
Reduced LP has 34 rows, 56 columns, and 170 nonzeros.
Presolve time = 0.00 sec. (0.07 ticks)
Symmetry aggregator did 52 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =          1000.000000

Dual crossover.
  Dual:  Fixing 4 variables.
        3 DMoves:  Infeasibility  0.00000000e+00  Objective  4.50000000e+03
        0 DMoves:  Infeasibility  0.00000000e+00  Objective  4.50000000e+03
  Dual:  Pushed 0, exchanged 4.
  Primal:  Fixed no variables.
* model BRT solved with objective = 4500.000


In [24]:
from docplex.mp.model import Model
master_mdl = Model('Master-Cutstock')
dummy_vars = master_mdl.continuous_var_dict(items, lb=0, ub=max_cut, name="dummy_cut")
cost = master_mdl.sum( 1000*dummy_vars[i] for i in items)
obj = master_mdl.minimize( cost )
cts = master_mdl.add_constraints( dummy_vars[i]>= item_demands[i] for i in items)
master_mdl.solve()
master_mdl.report()

NameError: name 'items' is not defined

In [63]:
for j in range (numRutas):
    if(y[j].solution_value):
        print(rutas[j])   
        print(y[j].solution_value)
        

(1, 1, 1)
0.5
(1, 0, 1)
0.5


In [48]:
matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
r={}
for i in matriz:
    r[i]=0
    



mdlAux= Model("Aux")
rutaNueva=mdlAux.binary_var_dict(matriz,name="r")

dualCapacidad = mdl.dual_values(capacidad)



objAux=mdlAux.maximize(mdlAux.sum((-rutaNueva[i,j])*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))


mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=1)

for i in range(1,numNodos):    
    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1)))
    
for i in range(numNodos):
    for j in range(i+1,numNodos):
        mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])
        
for i in range(numNodos):
    mdlAux.add_constraint(rutaNueva[(i,i)]==0)
                       
mdlAux.solve(log_output=True)
mdlAux.report()    

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 7 rows and 7 columns.
Aggregator did 2 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)
* model Aux solved with objective = 6.000
